<a href="https://colab.research.google.com/github/Beebruna/API-TRAINING/blob/main/doe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação python 3.10

In [ ]:
!sudo apt install python3.10

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10-minimal
Suggested packages:
  python3.10-venv binfmt-support
The following NEW packages will be installed:
  libpython3.10-minimal libpython3.10-stdlib python3.10 python3.10-minimal
0 upgraded, 4 newly installed, 0 to remove and 20 not upgraded.
Need to get 5,093 kB of archives.
After this operation, 19.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.10-minimal amd64 3.10.6-1+bionic1 [823 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.10-minimal amd64 3.10.6-1+bionic1 [1,968 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bion

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

In [ ]:
!sudo update-alternatives --config python3

There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.7    2         auto mode
  1            /usr/bin/python3.10   1         manual mode
  2            /usr/bin/python3.6    1         manual mode
  3            /usr/bin/python3.7    2         manual mode

Press <enter> to keep the current choice[*], or type selection number: 1
update-alternatives: using /usr/bin/python3.10 to provide /usr/bin/python3 (python3) in manual mode


In [ ]:
!python3 --version

Python 3.10.6


# DOE

In [ ]:
import requests
from tqdm import tqdm
import gdown
import time
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def get_html(url: str) -> BeautifulSoup:
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
               'referer':'https://www.ioepa.com.br/Frame',
               'accept-encoding': 'gzip, deflate, br',
               'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
               'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
               ':path': f'{url.removeprefix("https://www.ioepa.com.br")}'
               }
    html = requests.get(url, headers=headers)
    print(html)
    dom = BeautifulSoup(html.text, 'html.parser')
    return dom

In [ ]:
def get_link(html: BeautifulSoup) -> str:
    return html.find('div', id= "mid").a['href']

In [ ]:
def verify_doe(html: BeautifulSoup) -> bool:
    status = html.find('div', class_="alert alert-danger")
    return bool(status)

In [ ]:
def download_file_from_google_drive(link, destination):
    URL = "https://docs.google.com/uc?export=download"

    id = id = link.split('/')[5]

    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    print(response)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_file(response, destination)

In [ ]:
def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

In [ ]:
def save_file(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
        print('download: ', response.url)

In [ ]:
dates = pd.date_range("20210101", pd.Timestamp.today())

In [ ]:
datas = list()
for data in tqdm(dates):
    html = get_html('https://www.ioepa.com.br/Frame/?dts=' + str(data.date()))

    if verify_doe(html): #NONE = existe o documento
        datas.append(data)
        continue

    link = get_link(html)
    
    #save_file(link, str(data.date()) + '.pdf')
    print(data)
    download_file_from_google_drive(link, f'/content/drive/MyDrive/Meus Projetos/doc_doe/{data}.pdf')
    time.sleep(5)

  0%|          | 0/606 [00:00<?, ?it/s]


AttributeError: ignored

In [ ]:
'https://www.ioepa.com.br/Frame/?dts='.removeprefix('https://www.ioepa.com.br')

AttributeError: ignored

In [ ]:
!rm -f 2021

rm: cannot remove 'drive': Is a directory
rm: cannot remove 'sample_data': Is a directory


In [ ]:
!python --version

Python 3.7.13
